# Lab Exercise 4

Exercise description in German.

## Vergleich: Kalman-, ROSE- und Partikel-Filter

### Problemstellung

Gegeben ist die unten beschriebene zeitliche Folge. Diese kann z.B. als Position eines Fahrzeugs interpretiert werden.

In [ ]:
%matplotlib widget
import pandas as pd

beta_filter_data = pd.read_csv(
    "data/BetaFilter_data_in.dat",
    sep=" ",
)
beta_filter_data

In [ ]:
beta_filter_data.plot(
    x="time",
    y="position",
    kind="scatter",
    grid=True,
    s=1,
    xlabel="Zeit [s]",
    ylabel="Position [m]",
    title="Eingangsdaten",
    subplots=True,
    figsize=(10, 4),
)

### Aufgabe: Kalman-Filter

Realisieren Sie hierfür ein Beta-Filter, welches die Daten filtert und die Geschwindigkeit schätzt. Basis ist das folgende Beta-Filter. Hierzu müssen Sie noch die Varianz des Mess- und Systemrauschens abschätzen.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

y = beta_filter_data["position"].to_numpy()
u = np.zeros((y.size, 1))
t = beta_filter_data["time"].to_numpy()

Ts = t[1] - t[0]

In [ ]:
stable_s = y[t < 2]
R = stable_s.var()
Q = (6 / 80) ** 2
print(f"{R=:g}, {Q=:g}")

Ad = np.array([[1, Ts], [0, 1]])
Bd = np.array([[0], [0]])
C = np.array([[1, 0]])
D = 0
Gd = np.array([[Ts], [1]])

In [ ]:
x_hat = np.array([[y[0]], [0]])
P_hat = 50 * np.eye(2)

d_y = np.zeros_like(y)

s = np.zeros_like(y)
v = np.zeros_like(y)
K1 = np.zeros_like(y)
K2 = np.zeros_like(y)
P_hat1 = np.zeros_like(y)
P_hat2 = np.zeros_like(y)
P_hat3 = np.zeros_like(y)
P_hat4 = np.zeros_like(y)
for i, y_i in enumerate(y):
    if np.isnan(y_i):
        x_tilde = x_hat
        P_tilde = P_hat
    else:
        d_y[i] = y_i - (C @ x_hat + D * u[i])
        k = C @ P_hat @ C.T + R
        if k.ndim >= 2:
            K = P_hat @ C.T @ np.linalg.pinv(k)
        else:
            K = P_hat @ C.T / k
        x_tilde = x_hat + (K * d_y[i]).reshape(-1, 1)
        P_tilde = (np.eye(Bd.size) - K @ C) @ P_hat

    x_hat = Ad @ x_tilde + (Bd @ u[i]).reshape(-1, 1)
    P_hat = Ad @ P_tilde @ Ad.T + (Gd * Q) @ Gd.T

    s[i], v[i] = x_tilde
    K1[i], K2[i] = K
    P_hat1[i], P_hat2[i] = P_hat[0, :]
    P_hat3[i], P_hat4[i] = P_hat[1, :]
fig, (ax_s, ax_v) = plt.subplots(2, 1, figsize=(15, 7))

ax_s.plot(t, y, "k*")
ax_s.plot(t, s, "b--")
ax_s.plot(t, s + 3 * np.sqrt(P_hat1), "b-")
ax_s.plot(t, s - 3 * np.sqrt(P_hat1), "b-")
ax_s.grid(True)
ax_s.set_xlabel("Zeit [s]")
ax_s.set_ylabel("Position [m]")

ax_v.plot(t, v, "r-*")
ax_v.grid(True)
ax_v.set_xlabel("Zeit [s]")
ax_v.set_ylabel("Geschwindigkeit [m/s]")

fig.suptitle(r"Kalman-Filter: $\beta$")
fig.tight_layout()

### Aufgabe: ROSE-Filter

Nun soll für das selbe Problem ein ROSE-Filter entwickelt werden. Basis ist wieder ein Beta-Filter.

#### Messrauschen

Die Schätzung des Messrauschens lässt sich über den folgenden Zusammenhang bestimmen:

$$
\begin{align}
\boldsymbol{R}(k) & = \mathrm{E}{\left( {\left( \boldsymbol{y}(k) - \mathrm{E}{\left( \boldsymbol{y}(k) \right)} \right)} \cdot {\left( \boldsymbol{y}(k) - \mathrm{E}{\left( \boldsymbol{y}(k) \right)} \right)}^T \right)} \\
& \approx \mathrm{E}{\left( {\left( \boldsymbol{y}(k) - \boldsymbol{\hat{y}}_R(k) \right)} \cdot {\left( \boldsymbol{y}(k) - \boldsymbol{\hat{y}}_R(k) \right)}^T \right)} \\
& \approx \mathrm{E}{\left(  {\Delta \boldsymbol{\hat{y}}_R(k)} \cdot {\Delta \boldsymbol{\hat{y}}_R(k)}^T \right)} \\
\end{align}
$$

Die Berechnung von $\mathrm{E}{\left( \boldsymbol{y}(k) \right)}$ geschieht über ein Beta-Filter mit fester Verstärkung und die Berechnung von $\mathrm{E}{\left(  {\Delta \boldsymbol{\hat{y}}_R(k)} \cdot {\Delta \boldsymbol{\hat{y}}_R(k)}^T \right)}$ mit einem Alpha-Filter auch mit fester Verstärkung.

Da für das Beta-Filter die Größen $R$ und $Q$ zeitinvariant sind, konvergiert die Kalman-Verstärkung auf einen festen Wert, der sich mit der Gleichung

$$
\begin{align}
K &= \frac{0.125}{T_s} \cdot \begin{bmatrix}
T_s {\left( -\lambda^2 - 8 \lambda + (\lambda + 4) \cdot \sqrt{\lambda^2 + 8\lambda} \right)}\\
2 {\left( \lambda^2 + 4 \lambda - \lambda \cdot \sqrt{\lambda^2 + 8 \lambda} \right)} \\
\end{bmatrix} \\
\text{mit:} \quad \lambda & = T_s \cdot \sqrt{\frac{Q}{R}} = T_s \cdot \sqrt{\frac{\mathrm{Var}{\left( z(k) \right)}}{\mathrm{Var}{\left( v(k) \right)}}}
\end{align}
$$

bestimmen lässt [1].

Bei der Wahl des Verhältnisses von $Q / R$ ist man frei. Aufgrund der großen Dynamik sollte man das Verhältnis jedoch nicht zu klein wählen. Sinnvoll ist z.B. eine Wahl von $Q / R = 1$.

Da bei den Kalman-Filtern mit fester Kalman-Verstärkung nur die geschätzte Zustandsgröße für die weitere Verarbeitung relevant ist, müssen nur die folgenden beiden Gleichungen betrachtet werden:

$$
\begin{align}
\boldsymbol{\hat{x}}(k) & = \boldsymbol{A}_d \cdot \boldsymbol{\tilde{x}}(k - 1) + \boldsymbol{B}_d \cdot \boldsymbol{u}(k - 1) \\
\boldsymbol{\tilde{x}}(k) & = \boldsymbol{\hat{x}}(k) + \boldsymbol{K} \cdot {\left( \boldsymbol{y}(k) - \boldsymbol{C} \cdot \boldsymbol{\hat{x}}(k) - \boldsymbol{D} \boldsymbol{u}(k ) \right)} \\
\end{align}
$$

Berücksichtigt man, dass $\boldsymbol{B}_d = \begin{bmatrix}0 & 0\end{bmatrix}^T$ und $\boldsymbol{D} = \begin{bmatrix}0 & 0\end{bmatrix}^T$ und setzt die vorherige Gleichung (1) in (2) ein, folgt:

$$
\begin{align}
\boldsymbol{\tilde{x}}(k) & = \boldsymbol{\hat{x}}(k) + \boldsymbol{K} \cdot \boldsymbol{y}(k) - \boldsymbol{K} \cdot \boldsymbol{C} \cdot \boldsymbol{\hat{x}}(k) \\
& = \boldsymbol{K} \cdot \boldsymbol{y}(k) + {\left( \boldsymbol{I} - \boldsymbol{K} \cdot \boldsymbol{C} \right)} \cdot \boldsymbol{\hat{x}}(k) \\
& = \boldsymbol{K} \cdot \boldsymbol{y}(k) + \underbrace{{\left( \boldsymbol{I} - \boldsymbol{K} \cdot \boldsymbol{C} \right)} \cdot \boldsymbol{A}_d}_{\boldsymbol{H}} \cdot \boldsymbol{\tilde{x}}(k - 1)
\end{align}
$$

Mit diesem Filter wird der Erwartungswert $\boldsymbol{\hat{y}}_R(k) = \mathrm{E}{\left( \boldsymbol{y}(k) \right)}$ für die gemessene Position abgeschätzt.

$$
\begin{align}
\boldsymbol{\tilde{x}}(k) & = \begin{bmatrix}
\tilde{x}_1(k) \\
\tilde{x}_2(k) \\
\end{bmatrix} = \boldsymbol{K} \cdot \boldsymbol{y}(k) + \boldsymbol{H} \cdot \boldsymbol{\tilde{x}}(k - 1) \\
\hat{y}_R(k) & = \begin{bmatrix}
1 & 0 \\
\end{bmatrix} \cdot \boldsymbol{\tilde{x}}(k) = \tilde{x}_1(k)
\end{align}
$$

Zur Schätzung der Kovarianz des Messrauschens $\boldsymbol{R}(k)$ ist es aufgrund der geringen Dynamik der Größe $\boldsymbol{\hat{y}}(k)$ ausreichend, Rden Erwartungswert von $\boldsymbol{R}(k)$ approximativ über einen Alpha-Filter abzuschätzen:

$$\boldsymbol{R}(k) = \gamma \cdot \alpha_R \cdot {\left( \boldsymbol{\hat{y}}_R(k) - \boldsymbol{y}(k) \right)} \cdot {\left( \boldsymbol{\hat{y}}_R(k) - \boldsymbol{y}(k) \right)}^T + {\left( 1 - \alpha_R \right)} \cdot \boldsymbol{R}(k - 1)$$

Der frei wählbare Verstärkungsfaktor γ dient als Korrekturfaktor, da die Kovarianz des Messrauschens oft zu klein geschätzt wird. Aufgrund dessen sind Werte von $\gamma > 1$ sinnvoll.

#### Systemrauschen

Zur Schätzung der Kovarianz des Systemrauschens ist es notwendig, die Hilfsgröße $\boldsymbol{M}(k)$ zu bestimmen. Diese wird definiert durch:

$$
\begin{align}
\boldsymbol{M}(k) & = \mathrm{E}{ \left( {\Delta \boldsymbol{\hat{y}}_R(k)} \cdot {\Delta \boldsymbol{\hat{y}}_R(k)}^T \right)} \\
\text{mit} \quad & {\Delta \boldsymbol{y}(k)} = \boldsymbol{y}(k) - \boldsymbol{C} \cdot \boldsymbol{\hat{x}}(k) - \boldsymbol{D} \cdot \boldsymbol{u}(k) \\
\end{align}
$$

Der Erwartungswert kann durch ein Alpha-Filter angenähert werden. Dieses wird durch folgende Gleichung beschrieben:

$$\boldsymbol{M}(k) = \alpha_M \cdot {\Delta \boldsymbol{y}(k)} \cdot {\Delta \boldsymbol{y}(k)}^T + {\left( 1 - \alpha_M \right)} \cdot \boldsymbol{M}(k - 1)$$

Die Kovarianz des Messrauschens $Q(k - 1)$ lässt sich allgemein durch folgende Gleichung berechnen:

$$\boldsymbol{C} \cdot \boldsymbol{G}_d \cdot \boldsymbol{Q}(k - 1) \cdot \boldsymbol{G}_d^T \cdot \boldsymbol{C} = \boldsymbol{M}(k) - \boldsymbol{R}(k) - \boldsymbol{C} \cdot \boldsymbol{A}_d \cdot \boldsymbol{\tilde{P}}(k - 1) \cdot \boldsymbol{A}_d^T \cdot \boldsymbol{C}^T$$

Bestimmen Sie den linken Teil der Gleichung für ein Beta-Filter mit $\boldsymbol{G}_d^T = \begin{bmatrix} T_s & 1 \end{bmatrix}$ und $\boldsymbol{C} = \begin{bmatrix} 1 & 0 \end{bmatrix}$ und lösen Sie die Gleichung nach $Q(k − 1)$ auf.

Nach kurzer Rechnung folgt:

$$Q(k - 1) = \frac{\boldsymbol{M}(k) - \boldsymbol{R}(k) - \boldsymbol{C} \cdot \boldsymbol{A}_d \cdot \boldsymbol{\tilde{P}}(k - 1) \cdot \boldsymbol{A}_d^T \cdot \boldsymbol{C}^T}{T_s^2}$$

Ergänzen Sie nun in dem Programm die Zeile mit der Berechnung von $Q(k)$ und legen Sie die Größen $Q_{\mathrm{min}}$ und $Q_{\mathrm{max}}$ fest. Gute Werte sind eine Zehnerpotenz größer bzw. eine Zehnerpotenz größer als der Wert bei einem klassischen Kalman-Filter (siehe vorherige Aufgabe).

In [ ]:
y = beta_filter_data["position"].to_numpy()
u = np.zeros((y.size, 1))
t = beta_filter_data["time"].to_numpy()

Ts = t[1] - t[0]

Gamma = 2.5
Alpha_R = 0.05
Alpha_M = 0.1
Q_min = ...
Q_max = ...

Ad = np.array([[1, Ts], [0, 1]])
C = np.array([[1, 0]])

R0 = 1
Q0 = 1

lambd = Ts * np.sqrt(Q0 / R0)

K1 = -1 / 8 * (lambd**2 + 8 * lambd - (lambd + 4) * np.sqrt(lambd**2 + 8 * lambd))
K2 = 0.25 * (lambd**2 + 4 * lambd - lambd * np.sqrt(lambd**2 + 8 * lambd)) / Ts

K0 = np.array([[K1], [K2]])
H = (np.eye(Ad.shape[0]) - K0 @ C) @ Ad

In [ ]:
R = 1

Ad = np.array([[1, Ts], [0, 1]])
Bd = np.array([[0], [0]])
C = np.array([[1, 0]])
D = 0
Gd = np.array([[Ts], [1]])

GG = Gd @ Gd.T

x_hat = np.array([[y[0]], [0]])
x1 = x_hat

u = np.zeros((y.size, 1))
P_tilde = 1 * np.eye(2)
M = C @ P_tilde @ C.T

d_y = np.zeros_like(y)
yr = np.zeros_like(y)
Q = np.zeros_like(y)

s = np.zeros_like(y)
v = np.zeros_like(y)
K1 = np.zeros_like(y)
K2 = np.zeros_like(y)
M1 = np.zeros_like(y)
R1 = np.zeros_like(y)
P_hat1 = np.zeros_like(y)
P_hat2 = np.zeros_like(y)
P_hat3 = np.zeros_like(y)
P_hat4 = np.zeros_like(y)
P_tilde1 = np.zeros_like(y)
P_tilde2 = np.zeros_like(y)
P_tilde3 = np.zeros_like(y)
P_tilde4 = np.zeros_like(y)
for i, y_i in enumerate(y):
    x1 = H @ x1 + K0 * y_i
    yr[i] = x1[0]
    R = (
        Gamma
        * Alpha_R
        * np.array(
            [y_i - x1[0]] @ np.array([y_i - x1[0]]).reshape([-1, 1]) + (1 - Alpha_R) * R
        )
    )
    d_y[i] = y_i - (C @ x_hat + D * u[i])
    M = Alpha_M * d_y[i] * d_y[i].T + (1 - Alpha_M) * M

    Q[i] = ...

    if Q[i] < Q_min:
        Q[i] = Q_min
    elif Q[i] > Q_max:
        Q[i] = Q_max

    P_hat = Ad @ P_tilde @ Ad.T + GG * Q[i]

    k = C @ P_hat @ C.T + R
    if k.ndim >= 2:
        K = P_hat @ C.T @ np.linalg.pinv(k)
    else:
        K = P_hat @ C.T / k
    x_tilde = x_hat + (K * d_y[i]).reshape(-1, 1)
    P_tilde = (np.eye(Bd.shape[0]) - K @ C) @ P_hat
    x_hat = Ad @ x_tilde + (Bd @ u[i]).reshape(-1, 1)

    s[i], v[i] = x_tilde
    M1[k] = M(1)
    K1[i], K2[i] = K
    R1[k] = R(1)
    P_hat1[i], P_hat2[i] = P_hat[0, :]
    P_hat3[i], P_hat4[i] = P_hat[1, :]
    P_tilde1[i], P_tilde2[i] = P_tilde[0, :]
    P_tilde3[i], P_tilde4[i] = P_tilde[1, :]
fig, (ax_s, ax_v, ax_R1, ax_Q) = plt.subplots(4, 1, figsize=(15, 7))

ax_s.plot(t, y, "k*")
ax_s.plot(t, s, "b--")
ax_s.plot(t, s + 3 * np.sqrt(P_tilde1), "b-")
ax_s.plot(t, s - 3 * np.sqrt(P_tilde1), "b-")
ax_s.grid(True)
ax_s.set_xlabel("Zeit [s]")
ax_s.set_ylabel("Position [m]")

ax_v.plot(t, v, "r-*")
ax_v.grid(True)
ax_v.set_xlabel("Zeit [s]")
ax_v.set_ylabel("Geschwindigkeit [m/s]")

ax_R1.plot(t, R1)
ax_R1.grid(True)
ax_R1.set_xlabel("Zeit [s]")
ax_R1.set_ylabel("R")

ax_Q.plot(t, Q)
ax_Q.grid(True)
ax_Q.set_xlabel("Zeit [s]")
ax_Q.set_ylabel("Q")

fig.suptitle(r"ROSE-Filter: $\beta$")
fig.tight_layout()

[1] BAR -S HALOM, Yaakov ; L I, Xiao-Rong: Estimation and tracking: Principles Techniques and Software. Boston : Artech House, 1993. – ISBN 0 – 89006 – 643 – 4